## Поиск данных которые хорошо разделяются на кластеры с помощью Kmeans

In [4]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go 

palette = px.colors.sequential.Brwnyl
file_name = '~/MSU ML/HW_10_data/' + input()+ '.csv'
data = pd.read_csv(file_name,sep = ',')
data = data.values


fig_1 = sp.make_subplots(rows=2, cols=1, subplot_titles= ('Given', 'Predicted'), vertical_spacing= 0.1)
fig_1.add_trace(go.Scatter(
    x = data[:,0],y  = data[:,1],mode ='markers',marker= dict(color = data[:,2])),
    row = 1,
    col = 1)

fig_1.add_trace(go.Scatter(
    x = data[:,0],y  = data[:,1],mode ='markers'),
    row = 2,
    col = 1)
fig_1.update_layout(height=1000, width=800)
fig_1.show()

N = int(input())
model = KMeans(n_clusters=N)
prediction = model.fit_predict(data[:,0:2])


px.scatter(x = data[:,0],y  = data[:,1],color= prediction, color_continuous_scale= palette)


In [149]:
import numpy as np
def get_distance(X: np.array, center: np.array)-> np.array:
    distance = np.sqrt(np.sum(pow(X - center,2),axis= 1))
    return distance.reshape(-1,1)
        

def get_new_centers(X:np.array, Y: np.array, centers: np.array):
    clusters = np.hstack((X,np.argmin(Y, axis = 1).reshape(-1,1))) # Ищем какому кластеру принадлежат объекты
    
    new_centers = np.mean(X[clusters[:,-1] == 0],axis = 0) #Рассчитываем новые центры кластеров
    for i in range(1,centers.shape[0]):
        new_centers = np.vstack((new_centers,np.mean(X[clusters[:,-1] == i],axis = 0)))
        
    change = np.max(get_distance(centers, new_centers),axis = 0)# Вычисляем  смещение центров
    return new_centers,change



class KMeans(object):
    def __init__(self, K: int, init: np.array) -> None:
        self.K = K
        self.init = np.array(init)
        self.M = np.array(init).shape[0]
    def fit(self, X: np.array) -> None:
        count = 0
        distances = get_distance(X,self.init[0,:])
        for i in range(1,self.K): # Вычисляем дистанции для каждого объекта до каждого центра
            distances = np.hstack((distances,get_distance(X, self.init[i,:])))
        
        
        self.init,change  = get_new_centers(X,distances,self.init)
        while change >= 0.001 and count < 100: # Условие остановки максимальное смещение < 0.001 или количство итераций больше 100
            count += 1
        
            distances = get_distance(X,self.init[0,:])
            for i in range(1,self.K): # Вычисляем дистанции для каждого объекта до каждого центра
                distances = np.hstack((distances,get_distance(X, self.init[i,:])))
            
            self.init,change  = get_new_centers(X,distances,self.init)
        return self.init
    def predict(self, X: np.array) -> np.array:
        fin_dist = get_distance(X,self.init[0,:])
        for i in range(1,self.M): # Вычисляем дистанции для каждого объекта до каждого центра
               fin_dist = np.hstack((fin_dist,get_distance(X, self.init[i,:])))
        answer = np.argmin(fin_dist, axis = 1)
        return np.array(answer)

In [154]:
M = 4
N = 2
X = np.random.randn(N, M)
X
start = np.array([[40,0],
         [20,40],
        #  [90,40],
         [70,70]
         ])
start.shape
model = KMeans(K = 3, init= start)
model.fit(data[:,0:2])
print(model.predict(data[:,0:2]))

fig = go.Figure()
fig.add_trace(go.Scatter( x = data[:,0],y  = data[:,1],mode ='markers' ,marker = dict(color = model.predict(data[:,0:2]))))
fig.add_trace(go.Scatter(x = model.init[:,0],y = model.init[:,1],mode = 'markers'))


[1 1 1 ... 2 2 2]
